This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_target_histogram(y_df, savepath, title='target histogram', label='target values'):
    """
    Method to plot the histogram of true y values

    Args:

        y_df: (pandas dataframe), dataframe of true y data values

        savepath: (str), path to save the plotted precision-recall curve

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.70
    fig, ax = make_fig_ax(aspect_ratio=0.5, x_align=x_align)

    #ax.set_title(title)

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=y_df)

    # do the actual plotting
    ax.hist(y_df, bins=num_bins, color='b', edgecolor='k')#, histtype='stepfilled')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))


    plot_stats(fig, dict(y_df.describe()), x_align=x_align, y_align=0.90, fontsize=14)
    # Save input data stats to csv
    savepath_parse = savepath.split('target_histogram.png')[0]
    y_df.describe().to_csv(savepath_parse+'/''input_data_statistics.csv')

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_df = pd.Series(pd.read_csv(StringIO('''
14.917456294166673
14.190715794166673
13.833448794166669
14.237675928333335
14.37592540366667
13.513528678333335
14.257288717142863
13.601997967142859
13.409355717142864
14.418312261666667
13.796526761666668
13.479810011666668
15.333474936166668
14.719723186166663
14.409747686166662
14.657330570333333
14.273504820333333
14.118447320333333
14.670598609142857
14.260354109142856
14.003273859142856
14.83708065366667
14.32002315366667
14.056525903666673
6.829578057000001
12.786665821999998
12.786763821999998
12.790966821999998
12.765398572
14.927820024142857
14.738010774142857
7.944088417000001
14.619303931999998
14.490905932
14.296470431999998
14.033771932
9.67727495
18.053911715
17.684074715
16.925374465
16.127878965
7.9845418575
14.636635872499998
14.511181872499998
14.325286122499998
14.0670763725
7.656968625000001
20.714522975
20.755321975
20.850006725
14.58453414
20.545194225
20.591697975
20.687373225
14.422515140000002
20.304268475
14.13495964
13.824523890000002
7.010839242
12.412569507
12.495426007
12.672093007
12.791643757000001
7.6433469515
14.4410769665
14.2970867165
14.0535119665
13.771901216500002
7.584915452000001
14.398197967
14.252375967
14.001472717
13.718779967
7.74933381075
14.506242825749998
14.36747657575
14.14065732575
13.86475757575
6.4565564470000005
20.191390452
20.127357952
12.397995212000005
12.396516712000006
12.394971462000004
12.382044212000006
6.814702289000001
13.807504266000002
13.773859016000007
13.699270516000002
13.613922516
21.850246655333336
5.692056455
11.585613932000001
11.708253182
11.956653682
12.180632432000001
6.117558747000003
12.438579224000005
12.495853474000006
12.612122724000006
12.729608724000006
7.3828952091666675
16.228109686166672
15.931329936166664
15.333791936166664
6.38745748
13.289552707000002
13.206571457
13.078814707000001
6.280911190000001
13.595905167000003
13.430178417000006
13.232431417000004
7.252073593333334
15.530915070333332
15.242690070333333
14.658225320333333
6.6224156533333325
14.056311380333335
13.883536630333335
13.625775130333336
5.7492273475000015
11.923701324500005
11.988477074500002
12.110592574500005
5.917080846428571
12.229154823428571
12.250186823428571
12.290864573428571
6.57593423375
14.471903460750001
14.18747871075
13.77935646075
6.10207411
12.825036337
12.794372337
12.743253337
6.3742782983333335
13.601439525333335
13.489389775333333
13.289479525333336
7.113896882142859
15.757826859142856
15.398500109142857
14.671707109142856
6.628725668333333
14.285880645333336
14.079857145333335
13.773658895333336
7.682102753
15.31257448
15.15768673
14.87070148
5.480489952500001
11.0494934295
11.2037681795
11.5276056795
11.8469016795
7.0745554266666675
15.598046653666671
15.34589565366667
15.09128190366667
14.83707565366667
14.32002065366667
14.05652315366667
6.347188742
20.461667488666667
20.777828488666664
21.354218488666667
19.568985843666663
19.95102509366666
20.688970343666664
24.18993309783333
24.144550597833327
24.05295134783333
21.597848828666674
21.749433078666673
22.075505828666667
23.492781481999998
23.506125731999997
23.550854731999998
19.320838583666664
19.718128333666662
20.502742833666662
23.71859102080953
23.65938477080953
23.59076252080953
21.86553914533333
21.98437064533333
19.036981091166663
19.45374859116666
20.289171841166667
14.04248888916667
13.143963139166667
12.69146813916667
13.345876273333335
14.485312070333332
12.358682773333335
13.36064731214286
12.515053812142858
12.241403562142859
13.544733356666667
12.744566356666667
12.338379356666668
12.835545759
19.854120486
19.818714235999998
19.752915736
19.651199736
11.737439675
17.633718902000002
17.757594151999996
18.008456402
18.229608152
12.175249717
18.487073444000004
18.544612694
18.663392694000002
18.777974194000002
11.5479781725
17.0998108995
17.257941649499998
17.5818636495
17.9150451495
19.514434115
13.525789748
13.875999498000002
14.017717313
14.310910063
15.728321422999999
15.739101923
18.543885206
18.098046705999998
15.344545863499999
15.7502208635
15.5595494575
15.5867464575
15.62183681675
15.65007056675
13.643158453000002
13.952564703000002
15.106483352
15.004761102000003
14.739946852000001
13.969864960000002
13.920003960000004
13.836088210000005
12.932731497999999
12.981504248
13.088634747999999
15.034419223000002
14.926145723
14.709715223
12.232324662999996
12.296032162999998
12.435931162999998
7.286226983
14.325880032
14.666842282000001
14.794365096999998
15.088122096999998
""
16.509852207
19.35087999
18.89784674
16.517042897499998
16.5248678975
16.341789491500002
16.3618744915
16.39955735075
16.41485235075
14.442543737000001
14.751368987000001
14.287035852
14.422211352
14.685306352000001
14.867307244000001
14.942881744000001
15.093787494
13.813147031999998
13.980629781999998
14.325746281999994
15.963262757
15.993081507000001
16.046257507
13.129221947000001
13.342272947
13.714811697
8.075609767000001
13.724049890000002
14.053168390000002
14.068000205
14.215252705
14.501153205
15.912130315
15.921182315
18.761053097999998
18.300576348
15.9311752555
15.9362710055
15.751677099499998
15.772002349500001
15.80969995875
15.82555470875
13.689677095
13.846040595000002
14.157845345
13.614463710000004
13.662313210000002
13.78603746
15.341528744000001
15.311498744000001
15.093887994000001
13.126337140000002
13.228420640000005
13.429025390000005
15.291541615
15.285373865
15.131408365
12.456646055000002
12.578927555000002
12.821521055000002
7.485140125
13.443853895000002
13.804340645000002
13.93439396
14.26308996
15.595016070000002
15.602927570000002
18.871710102999998
18.416586353
15.6105855105
15.6180162605
15.4339126045
15.4533768545
15.49216721375
15.50701846375
13.5656036
13.911564599999998
13.255283715000004
13.261431215000005
13.274015465000002
14.981932749
14.856889249000002
14.540326620000002
13.836277107
13.788384857000002
13.693124357000002
13.187417145
13.223528395
13.296813395
14.95562937
14.83406162
14.640124620000002
12.108174560000002
12.190639810000002
12.35439656
7.168173379999999
14.458342652999999
14.779096153
14.952017717999999
15.222006468
16.681668828
16.691936327999993
19.518812111000003
19.063057361
16.697299018499994
16.707583768499994
16.5204833625
16.540642612499994
16.579845471749998
16.59632722175
14.581089608
14.725542358
14.876273608
14.427116973
14.534646223
14.702438973
16.172739757000002
16.154723757
16.115404757
15.026816865
15.084906115
15.202667365
13.987566903
14.106010403000006
14.368207653000002
13.272358318
13.428456817999999
13.746168317999999
8.254580137999998
12.956082093
13.325211592999999
13.447711408
13.769981408
15.128161768
15.152195017999999
17.690850050999998
""
15.1449192085
15.1706567085
14.9589530525
14.9811533025
15.02086041175
15.043542161749999
13.077258298
13.422490048
12.685998412999997
12.597824412999998
12.461528663
14.422677197
14.211217947
13.792296947
13.280567805
13.154003055
12.902376555
12.039801093
12.047015593
12.064521343
14.350894568
14.151469817999999
13.750844068
6.691342078
14.946229423499998
15.291998673499998
15.4456507385
15.777487988499999
17.105800598499993
17.1185508485
20.363253131499995
19.9101861315
17.121359788999992
17.134044289000002
16.944095133
16.965924883000007
17.002701492249994
17.02069574225
15.075369878499998
15.422942128499999
14.956671993500002
15.155955493500002
15.549278743500002
16.681478527499998
16.7463435275
16.8717522775
15.536330635499999
15.6792488855
15.962828135499999
14.8915019235
15.122244923499998
15.5806499235
16.6563096485
16.7250883985
16.9076761485
13.811333338499999
14.090143338499999
14.633877588499999
8.6763154085
14.763943759000005
14.954886009000006
15.260837074000001
15.417747574000005
17.115006684
17.132936433999998
20.311040967
19.859056967
17.131901124499993
17.148354624499998
16.946849218500002
16.9745784685
17.00892657775
17.03453232775
14.888040964000002
15.062761214000002
14.890804079
15.031799829
15.323364579000001
16.679881363
16.746766862999998
16.855310863
15.505497721000006
15.620901221000006
15.847917221000007
14.801289009000001
14.947632759000001
15.241548509000001
16.589123234000002
16.649790984000006
16.754526234000007
13.735003174000001
13.957675924
14.357918174000002
8.673783744000001
14.35741768825
14.626497938249994
14.85112050325
15.07177225325
16.61117886325
16.62248636325
19.39950164625
18.94093414625
16.62809155375
16.63831805375
16.44999589775
""
16.509114507
16.526688757000002
14.479219643249998
14.72484639325
14.39260025825
14.50449450825
14.70893200825
16.128250292249998
16.145994542249998
16.158297292249998
14.98362590025
15.068616150249998
15.231593650249998
13.89866268825
14.05756893825
14.37422993825
16.115419913249998
16.090264163249998
16.178588663249993
13.225886603249998
13.41935285325
13.761180603249993
8.18296442325
22.55562301916667
21.882070269166668
21.73123926916667
21.94431965333333
21.69845465333333
21.51781190333333
21.96117444214286
21.636054942142852
21.45161419214286
22.07348573666666
21.507270236666667
21.381057486666666
22.875904911166664
22.551523161166664
22.367288911166657
22.46257329533333
22.27058404533333
22.13848904533333
22.47241458414286
22.23845458414286
22.08881108414286
22.56373662866667
22.240339128666662
22.015176378666663
21.726396864166666
20.872081614166667
20.592563614166664
21.057009998333328
20.624646748333333
20.382819498333333
21.061915537142852
20.56250803714285
20.287296537142858
21.23491633166666
20.486287081666664
20.244540081666663
13.403051824999995
13.721534074999996
13.89178664
14.15817189
15.5912135
15.568389000000002
18.407400033
22.818078098
18.088358533
15.6031469405
15.5856599405
15.4250577845
15.4201622845
15.50084839375
13.91992388875
15.47537639375
13.522569529999998
13.821607279999998
14.999316678999994
14.884155678999994
14.624221928999994
13.870165036999998
13.815277786999998
13.722272786999998
12.827075575
12.876159075
12.978100325
14.93881355
14.8382883
14.59725905
12.115911489999998
12.17659399
12.375070739999998
7.18233006
14.265397358999994
14.578878858999994
14.744475173999998
15.001873424
16.434477534000003
16.419273284000003
19.217728566999995
18.742670567
16.4497029745
16.434085224500002
16.276166068499993
16.272524568499993
16.334292927750003
16.326924427749994
14.380468813999993
14.667297313999994
14.242835678999993
14.370664428999994
14.624235428999993
14.827152070999995
14.903543320999994
15.056204570999995
13.759769108999993
13.928440858999998
14.278240858999993
15.921846083999993
15.951554583999993
16.008488083999993
13.090890023999993
13.286157023999994
13.683353023999993
8.034305593999997
13.666990717000001
13.974852467
14.151912532000006
14.419681032000005
15.851769392000001
15.837528892
18.632009175
18.147372425
15.867105582499995
15.852451582499995
15.6926426765
15.689246676500002
15.75113903575
15.74432628575
13.785586172
14.074696922000001
13.580528286999998
13.631743786999998
13.738413536999998
15.306193070999994
15.222875570999994
15.056165820999995
13.091650716999998
13.196127717000001
13.388301466999998
15.255561692
15.203635692
15.095306692000001
12.425359381999993
12.546309381999993
12.790891631999994
7.450516951999999
13.384339972000001
13.722071722
13.873216287
14.182171037
15.527925896999998
15.513553396999994
18.73630668
18.25840068
15.543328587499996
15.528528837499994
15.368612681499998
15.3651974315
15.427120540749993
15.420273290749993
13.505732926999993
13.831489926999998
13.210560542
13.214100041999998
13.220698042
14.940292325999994
14.815257825999995
14.565501575999994
13.795312434000001
13.748233934000002
13.654749184000002
13.143467472000001
13.179574472
13.252101722
14.888301947
14.793695947
14.595937947000001
12.068158387
12.153464636999994
12.323642886999998
7.126342456999999
14.33493373
14.63346798
14.821945795
15.067936545000002
16.547180155000003
16.533592905000006
19.376727438
18.903184937999995
16.562583345500002
16.548860845500002
16.388417689500002
16.3849459395
16.446667548750003
16.43973804875
14.454491685
14.726992185
14.332304550000002
14.43989255
14.59721155
16.065643083999994
16.047936333999992
16.00849908399999
14.919574942
14.977913441999998
15.095192942
13.879589729999998
14.011631979999995
14.26445548
13.198852144999995
13.321973144999996
13.639539394999995
8.146407715
12.830370169999995
13.177351919999996
13.313832734999998
13.611457985
14.983948595
14.970130845
17.552593627999993
17.075221627999998
14.9991775355
14.9849932855
14.8250316295
14.821607129499998
14.88345123875
14.87698923875
12.948297874999998
13.272948124999996
12.57869224
12.49253099
12.37582174
14.311578023999994
14.115947273999994
13.697492523999994
13.171047881999993
13.043795631999993
12.791044381999994
11.93223367
11.939498919999998
11.956965169999998
14.240345394999995
14.040528894999996
13.639554894999995
6.582525154999997
14.873304000499994
15.193809000499995
15.371405315499999
15.681305315499998
17.028364425499998
17.0170514255
20.2346997085
19.7568464585
17.043868865999993
17.032437616
16.867706709999997
16.864815959999998
16.926860069249994
16.92136181925
15.001939705499995
15.325778955499995
14.904965320499995
15.102186570499995
15.492791820499995
16.6318573545
16.698162854499998
16.826436104499994
15.487319462499995
15.632410212499995
15.919953962499996
14.838635250499998
15.070474750499995
15.531294500499998
16.579933975499994
16.677442725499994
16.863510725499996
13.763855915499995
14.045920165499995
14.600219165499995
8.625199485499998
14.659995835999998
14.819478585999999
15.158733401000006
15.283263901000002
17.028205261000007
17.031802261000006
20.219364294000002
19.746347294
17.0451447015
17.050569701499995
16.8558552955
16.8550385455
16.92001115475
16.92027540475
14.784322791
14.924592790999998
14.817822155999998
14.947697406
15.248746906
16.590763189999997
16.64462394
16.778548439999998
15.432588548
15.527286797999999
15.774236047999999
14.728042585999999
14.852092586
15.160122335999999
16.517422311000008
16.554446061
16.688228061000007
13.664016750999998
13.846310750999999
14.276637750999999
8.610793071
14.248316015250001
14.48410051525
14.740391580250002
14.93574308025
16.481992190249994
16.469190190249993
19.30076947325
18.82454997325
16.497230630749993
16.483351630749993
16.34657997475
16.34552197475
16.409963083999994
16.407910333999993
14.370143470249998
14.58696247025
14.29545758525
14.424050085249998
14.64636858525
16.045474869249993
16.051049619249998
16.060980119249994
14.90451922725
14.98722497725
15.157114977249998
13.82111301525
13.96237426525
14.290479765250002
15.98843099025
16.02299874025
16.06538499025
13.146510430249998
13.302100430249993
13.660189180249994
8.089635000249999
13.470687624000002
13.381526874
13.229928124
12.106493082000004
12.083215832
12.116596082000001
11.726806398000006
11.760785398000007
11.824639898000006
14.913556110500007
14.703484360500006
14.341385360500002
14.961212837999998
14.763312337999995
14.391598337999998
13.547394829000002
13.463031079
13.308001079
12.203048537
12.203947287000004
12.229317787000001
11.851892102999999
11.893006352999999
11.987155852999999
15.0248105655
14.8315360655
14.455456565499999
15.096215292999997
14.915834792999997
14.563400292999996
19.487036094
19.421049344
19.268086844000003
18.129993552000002
18.140944552
18.183880302000002
17.777418118000003
17.808827618000006
17.888981368000003
20.959582330499998
20.759541830499998
20.393291830500004
21.011158058
20.820398558
20.453569557999998
19.509060094000002
19.413030094
19.262371344
18.150524552
18.136309052
18.181831802
17.783629618000003
17.821005118000006
17.899544868000003
20.960645580500003
20.765065830500003
20.3896375805
21.024118308
20.826579558
20.464748558
13.733415516000006
13.706523016000002
13.649203516000005
12.372549474000007
12.430336974000006
12.546771224000006
12.01300804
12.09076729
12.251119289999998
15.1953207525
15.0549900025
14.7847360025
15.25620248
15.12367273
14.84129748
13.831248721000007
13.796595721000006
13.731252221000002
12.477538929000001
12.535901929000001
12.657734929
12.136427245
12.230861995
12.419803495
15.3100479575
15.176060457499998
14.920631207499998
15.387205685
15.265157185
15.019103185
19.779619236
19.751121985999998
19.693166236000003
18.420542694
18.478178694
18.596164444000003
18.065081010000007
18.146441510000006
18.313355010000006
21.2454642225
21.1063917225
20.8420214725
21.3099307
21.16935245
20.9055002
19.774887736000004
19.744716986
19.686480486
18.417613694000003
18.475398444000003
18.594338444
18.066245260000006
18.151277760000006
18.324751510000006
21.2445167225
21.1068507225
20.8468177225
21.3125947
21.1767747
20.91672645
12.819840719
12.655351469000001
12.333384469
11.462069927
11.386057927000001
11.264851427
11.108478493000005
11.097924993000005
10.959690993000002
14.288278705500002
14.014701455500006
13.476678705500007
14.348473432999995
14.073644932999995
13.527578182999994
12.921429174000002
12.752824174
12.448993424000001
11.570871882
11.496648132
11.349467132000001
11.233454197999999
11.197341948
11.118236448
14.399356660499999
14.129384160499999
13.5842236605
14.477503637999995
14.220742637999995
13.687860637999995
18.867000439
18.702153439
18.383744939
17.510664397
17.435261646999994
17.311751397000002
17.160776963000007
17.112300463000008
17.015911713
20.3374209255
20.0648901755
19.524380675499998
20.401357652999998
20.130990652999998
19.586210153
18.864222189
18.698307189
18.382415689000002
17.509579897000002
17.434573397
17.287970897
17.163204963000002
17.119421213000006
17.030918963
20.3367829255
20.0651884255
19.5214944255
20.404415653
20.138166403
19.594335903
15.317630383333334
24.680866750476195
14.993871383333335
14.353133383333335
13.843178133333335
13.661357133333333
15.527539922142859
15.142765422142858
6.062420350000003
12.471375327
12.518253077
12.609803577000001
7.027681505
14.139814732
14.076935232000002
13.947302232000004
5.624955445000001
11.355235422
11.491120172
11.754019672
6.347283492000001
12.11233082
18.520537297
18.555411547000002
18.650090297000002
13.053618475000002
20.182317702
20.110366702
19.979491952
11.623148165
17.372201642
17.509820641999998
17.786483892
12.382068212000005
27.68596985
12.12276657
18.529821047000002
18.568290547
18.662265297
13.071140475000005
20.186694952000003
20.123490452000002
19.992938452
11.674823415
17.384894392
17.522142142
17.797797392
12.395003962000006
6.555534165000001
12.955709392000005
13.001457142000005
13.088301392000005
7.480849319999999
14.628863796999998
14.556619547
14.423789546999993
6.06627751
11.819428737000004
11.956717987000001
12.230790237000006
6.829647307
12.618188885
19.023689612000002
19.066820862
19.159281612
13.556225039999998
20.686029266999995
20.621699517
20.489843267
12.13566798
17.884047956999996
18.022058207
18.298879207
12.893945777
12.623166135
19.019204612000003
19.065482862000003
19.156983112
13.551395789999996
20.684824766999995
20.620062266999994
20.487853267000002
12.13404973
17.882611956999995
18.020707707
18.297203957
12.892532026999998
22.323745464
6.188414055
12.583403532000004
12.629937282
12.725911532000001
7.123292210000002
14.248504437000005
14.184720937000005
14.053092937000002
5.700417900000001
11.446892627
11.585023127000001
11.862569877
6.456604947000001
12.73602751025
12.74262276025
12.75587126025
13.25471257525
13.25391182525
13.252570325250002
15.120524185250002
15.01120793525
14.91014593525
18.56048121825
18.21707571825
17.76862046825
15.136253375749998
15.02809337575
14.93827612575
15.099369893249998
14.979403393250001
14.756625893249998
12.858741260250001
12.911729510249998
13.126550010250002
14.95189546975
14.84053071975
14.673810469749998
14.91353897025
14.798874220250001
14.623564220249998
15.014077329000001
14.902815579
14.758220829
15.779513483249998
15.621980233249998
15.425537483249993
14.34512026925
14.392920769249999
14.48161251925
13.030288580250001
13.17835433025
13.494770080250007
14.778132864249999
14.79831386425
14.83650961425
12.139575185250001
12.351819685250002
12.825842935250002
13.63049747225
13.72644897225
13.915949972249999
12.999517977249994
13.14371897725
13.434320727249998
13.381475888583331
13.490376138583331
13.711220388583332
16.829808939416672
16.660077939416663
16.306695439416668
13.93038996025
14.009374960250002
14.18278846025
12.935374010250001
13.08330076025
13.472407510250001
14.723465485250001
14.77388248525
14.87458323525
14.262927170249993
14.294332670249993
14.346908920249998
16.162963323583337
16.037986573583332
15.789480073583336
11.88650942525
12.09388467525
12.592607675250001
14.739602383583332
14.732666133583331
14.742736383583331
12.518700827750001
12.71032807775
13.09599007775
12.847427826678569
13.019458076678571
13.386764576678571
15.176588463999998
15.100427464000001
14.947847214000001
13.43126534025
13.539296840249998
13.762387090249998
14.246161778583332
14.28258702858333
14.381862778583331
15.31510625575
15.335926255750001
15.362596005750001
14.751862761916664
14.767583261916664
14.795201261916663
13.239388981916669
13.375850731916664
13.633650231916668
16.38756711239286
16.19641336239286
15.798004862392856
15.23311309125
15.21497784125
15.17974884125
15.905615233249993
15.869521233249998
15.769053483249998
11.59089618275
11.82282368275
12.382835432750001
16.204703656916667
16.079557406916667
15.817808406916669
6.816344245249997
14.022379587999994
14.425892087999998
14.472758403
14.860834902999999
16.160568013000002
16.255700262999998
18.962465295999998
18.594954546
16.1803257035
16.269462203499998
16.0225235475
16.1222700475
16.07467140675
16.17178540675
14.125434292999998
14.539111792999996
15.504434191999998
15.371266191999993
15.098276691999994
14.449211799999995
14.368307549999995
14.189613299999998
13.285915088
13.344594588
13.463543088
15.445126313
15.310480063
15.059293562999999
12.589522252999997
12.664765252999999
12.792871002999997
7.663357323
14.883270371999993
15.315989621999993
15.326471686999998
15.696421686999994
17.011083796999998
17.104740297
19.800538079999995
19.42937308
17.024866987499998
17.117981237499997
16.864768581499998
16.971101331499998
16.91839694075
17.02050544075
14.979195076999993
15.382599576999993
14.718443441999993
14.847956191999993
15.104860441999993
15.303394583999994
15.381848083999994
15.538497833999994
14.237289371999998
14.407403121999998
14.756722621999998
16.396495596999994
16.427612346999993
16.485977096999992
13.568129786999993
13.765082036999994
14.175409286999994
8.508517106999998
14.29122348
14.70859073
14.743807795000002
15.121070295000004
16.433658905
16.527422905
19.234779438
18.849098438
16.447659095499993
16.541053595499996
16.285556189500006
16.390088189500002
16.339911798750002
16.44104054875
14.392411435
14.794029685000002
14.061409299999998
14.106058049999998
14.214170299999996
15.787226833999993
15.704356583999994
15.538521083999994
13.582401479999998
13.679528979999995
13.869864729999998
15.736317205
15.685861955
15.577363205
12.907087394999996
13.029112144999996
13.273679144999996
7.930990964999999
13.996111735
14.444709985
14.46030255
14.8779758
16.105095159999998
16.198697159999995
19.318190443
18.948684443
16.118951350499998
16.212152850499994
15.957914694500001
16.0629041945
16.011858803749995
16.113239053749997
14.104869689999996
14.541873689999996
13.686835804999998
13.692313305
13.699172054999998
15.416601838999993
15.291659588999995
15.041717838999993
14.272488197000001
14.226307447
14.134302697
13.622613985
13.661597484999998
13.739446734999998
15.36432221
15.26904571
15.07312521
12.542921899999998
12.628139399999998
12.79306515
7.602494469999997
14.964332742999996
15.369943493
15.417648808
15.768560058000002
17.131573667999998
17.225183418
19.973762701
19.607322200999995
17.1457458585
17.239268608499998
16.982298702500003
17.0864724525
17.036990561750002
17.137513561749998
15.066581198
15.446707697999999
14.808640813
14.904478563
15.079833062999999
16.547712346999994
16.528507596999994
16.485913346999993
15.402950955
15.461204205
15.577330705
14.363103492999995
14.495692242999995
14.747213242999996
13.667322657999994
13.877399407999995
14.119586157999995
8.629025477999999
13.386995182999994
13.834472682999994
13.836301248
14.235409997999998
15.499482608
15.585640858
18.132577390999998
17.763150640999996
15.5132915485
15.5987802985
15.3530103925
15.4521861425
15.40598650175
15.50000350175
13.487440387999998
13.917123387999995
13.002155003
12.918911752999998
12.752111252999995
14.735277536999993
14.551396286999994
14.141279036999993
13.632557894999994
13.515971394999998
13.223476644999995
12.349955933
12.364062182999998
12.391720432999998
14.694739157999996
14.467801907999995
14.072648907999994
7.004438417999997
15.486249763499995
15.910658763499994
15.964928328499994
16.377507828499994
17.617314938499998
17.711597188499997
20.826706971499995
20.4543202215
17.631662878999997
17.725945628999995
17.465223472999998
17.567207973
17.521132582249994
17.62077158225
15.604982468499994
16.032503718499996
15.391662583499995
15.588525333499994
15.977124583499995
17.1200141175
17.1858421175
17.3130566175
15.975479475499995
16.120082225499996
16.406469975499995
15.3263815135
15.558943513499994
16.020570513499994
17.066854738499995
17.161236238499995
17.341898238499994
14.251078178499995
14.531156928499994
15.078959178499995
9.114095998499996
15.289612849000001
15.563238849000001
15.747045914000001
15.978545664
17.583448524
17.677398774000004
20.790963557
20.413517557
17.605872964499994
17.691907464499995
17.432072058499998
17.533717558499998
17.48759291775
17.58697166775
15.393332304000001
15.648615304000002
15.291869169000002
15.427887169000002
15.704513669
17.082325952999998
17.134762202999998
17.246224202999993
15.905665810999999
16.013694811
16.230665311
15.201563599000002
15.341559599000002
15.623428099000002
16.989788824
17.044887324
17.140382574
14.134211014000002
14.334112764
14.738088514000001
9.080457083999999
14.816958528250003
15.174994028250001
15.269042593250001
15.571190093250001
17.021035953249992
17.107613703249992
19.87116598625
19.49160423625
17.034937893749998
17.121296643749996
16.872811737750002
16.971732987750002
16.926131346999995
17.019931346999993
14.920202233249993
15.251297233249998
14.74187884825
14.85794034825
15.364742348250001
16.469314382249998
16.474312382249998
16.482668132249998
15.325329740250002
15.464667990250001
15.57569474025
14.25971452825
14.41569652825
14.736647028250001
16.46532700325
16.48370675325
16.51911900325
13.55639894325
13.754836943249998
14.07860594325
8.525266263249996
10.744103787500002
11.044677679500001
11.7123216925
10.2894031325
10.8952267875
20.405243231666667
19.721811615833328
19.875282404642856
19.391709633500003
19.810829449166672
11.2076289295
20.78819112366667
20.098070507833334
20.25183229664286
19.7957772755
20.287335841166662
11.8780961925
21.54659813666667
20.881487020833333
21.040238809642855
20.5517712885
20.960963354166665
10.4310198825
19.771066326666666
19.068016710833334
19.209210749642857
18.7521849785
19.151580794166673
18.0440715305
19.2430336855
19.5446665775
20.1943638405
12.1091751705
12.3729418125
13.059541825500002
11.529325515499997
12.3813676705
21.929316864666667
21.278557498833333
21.429637537642854
20.918952766500002
21.328962832166667
12.645587312500002
22.30051650666667
21.65332764083333
21.808727679642857
21.2460849085
21.69620422416667
13.3184683255
23.09237976966667
22.450748903833333
22.59858194264286
22.066485171500002
22.490092987166662
11.811655265499995
21.175138709666673
20.491411843833333
20.64641313264286
20.158616111500002
20.592979927166667
12.6668454205
12.9331795625
13.5989508255
12.102358265499998
11.9939369975
12.253807389499993
12.941370152499996
11.406739592499996
12.246557997499998
21.794684191666665
21.169657325833327
21.29250361464285
20.7731295935
21.216888409166668
12.513722139499993
22.15976708366666
21.560904217833333
21.758025256642856
21.1148917355
21.582879051166664
13.189845902499995
22.940995346666664
22.31474398083333
22.468401019642855
21.9357224985
22.35866931416666
11.677438842499996
21.041333036666664
20.360996420833327
20.514116959642852
20.0304044385
20.459951254166672
20.9117486455
21.180635537499995
21.8449425505
19.6487464905
10.868566992500002
11.1469656345
11.7937898975
10.3839645875
10.993729242500002
20.506230936666668
19.830037820833333
19.99323010964286
19.514613088500003
19.939946654166672
11.2818883845
20.88198032866667
20.19585996283333
20.34947825164286
19.879381230499998
20.299136796166664
11.9136598975
21.625997341666668
20.967506725833328
21.12316751464286
20.6272929935
21.042543809166666
10.506943087500002
19.860142031666665
19.157217165833327
19.312599454642857
18.842430183500003
19.263085499166664
19.364780390499998
19.6431487825
20.2733435455
18.8447452355
12.518307760499999
12.784361652499998
13.472893665499996
11.925382355499996
12.822205760499994
22.37377495466666
21.723903088833328
21.875588627642852
21.3468161065
21.786556672166668
13.091055652499998
22.76695934666667
22.22359098083333
22.29842376964286
21.6932362485
22.121751814166664
13.762950415499995
23.52400060966666
22.899878993833326
23.053058782642857
22.514982261500002
22.942996827166663
12.241320355499996
21.61534354966666
20.94085543383333
21.08897822264285
20.5956462015
21.035349267166666
21.589799908499998
21.867695050500004
22.524468313499995
21.0262470035
7.556447125
28.23664325783334
28.397976757833337
28.749864257833337
30.4587433345
30.5259898345
30.677511334499997
12.121709329500003
12.148069829500002
12.236745829500002
10.456974402000002
10.609470902000002
10.920482402000001
12.68317522866667
12.738822978666668
12.859127478666668
19.827213991166662
19.906248491166668
20.147400991166663
18.171570563666673
18.333746313666676
18.677056813666674
20.382972890333342
20.449181890333332
20.60362339033334
24.518815634499997
24.5346733845
18.7000158695
15.017532037142857
14.849641944999998
14.88760887575
17.590174477999998
14.867565885499998
14.779041153
14.674586229499994
14.63315673
14.60001998
14.740270588749995
15.367289992999996
20.920417565
20.10941767
21.453325457000002
19.854868409999995
27.85255912533334
27.648689285000003
27.92976970866666
18.750132797
21.52084179033333
17.1926631495
21.532999738666668
30.75049522666667
12.270475784500002
16.648379823
20.3627459775
20.3990139775
20.4789342275
18.6932028
18.848638800000003
19.15063655
20.904758126666668
20.983834876666673
20.946943876666673
20.534917975
20.527641540000005
20.23576518
20.38074643
19.333643285
19.4840756625
19.496277662499995
13.595017397499998
17.269160579999998
17.39322508
11.699141315
19.438635406666666
19.400684406666667
13.360008891666665
13.91703463875
12.767616782000001
15.055534025333333
14.4784703155
27.68596985
22.323745464
22.999367393666674
22.14855963
20.191390452
20.127357952
21.850246655333336
6.805748383999998
6.939458611000003
6.592801290250001
7.182541448999998
7.376550676000003
7.005484105250001
6.908835339
7.052906566000002
12.745536077
12.690706827
13.81774084
13.72446584
13.52675434
11.02002103
11.11761728
11.30205428
13.22121359
13.33751509
13.52578009
13.804545732000005
13.846812232000001
12.077582935
12.256329434999998
12.670809935
11.58550603
11.49602978
11.31783578
12.165099922000001
12.027487672000001
13.285379685
13.039333435
12.666327435
13.228275392000006
13.187734392000005
14.252799154999998
14.196220654999998
13.981766905
11.497949345
11.590306095
11.772873845
13.683591655
13.768241904999998
13.978712405
14.249685296999994
14.308433796999998
12.528975
12.724851
13.12511575
12.062103345
11.979628845
11.774509095
12.641469487000004
12.504824487000002
13.7228475
13.5236075
13.125466500000002
12.856858532
12.813392282
13.936147295
13.842325795
13.617887795
11.136880735000002
11.224850235000002
11.411420485
13.323018545
13.410792045
13.619661045
13.921758437000005
13.947163437000006
12.169560139999998
12.363710139999998
12.75041039
11.698703235000002
11.616449235
11.413775985
12.278703627
12.141458627
13.36021564
13.15795714
12.75400239
19.197092907000002
20.541318975
20.407413051999995
18.801442547
21.117181315
19.142367907
20.493140475
20.357177552
18.744482047
21.057684065
20.26906917
21.616307988000003
21.482371565
19.87461756
22.284049078000002
20.16293417
21.508655738
21.374658815000004
19.76950481
22.083841078000003
19.95217767
21.285899488
21.141281315000004
19.56382156
21.877855078000003
17.471884359999994
18.805688178
18.669671255
17.07917025
19.472550268
17.552891109999994
18.897647428
18.760057755
17.159703999999998
19.563609518
17.74107436
19.070461427999998
18.897085505
17.34819275
19.627909518
19.64862367
20.974399988
20.831151065000004
19.25839306
21.575184078000003
19.74741192
21.094576488
20.956559565000006
19.35650831
21.666941828000002
19.95147292
21.286082988
21.160366315000005
19.56025581
21.863485828
20.234239312
21.55927613
21.424312207
19.837633952
22.22930922
20.281433562
21.60963463
21.474634457
19.889587452
22.27960697
18.505362015
19.829858082999998
19.694526909999997
18.112632155
20.494576173
18.695116015
20.013178583
19.877685409999994
18.301137905
20.677301922999998
19.121027765
20.385823833
20.250261159999994
18.694805155
21.055230422999998
18.02573886
19.336918178
19.199744505
17.63171125
19.996962518
17.93703086
19.246440928
19.121558505
17.543948
19.903129768
17.742994109999994
19.055522928
18.918972755
17.35027775
19.635974518
18.605777502000002
19.91497782
19.778254647
18.210396892000002
20.578119159999996
18.469182252
19.77846707
19.642148896999995
18.073591642
20.44063866
19.694654515
20.994369332999998
20.857339409999994
19.300145155000006
21.659528172999998
19.482821265
20.793346333
20.673542659999995
19.088680905
21.459135422999992
19.076667765
20.388066083
20.252260909999997
18.681425405
21.056460923
19.188913157
21.006204290000003
20.812907867
19.292757112000004
21.51570313
19.161905407
20.96706479
20.833554617
19.259969362
21.47150813
20.25977792
22.052230553
21.90835463
20.355880875
22.602268393000003
20.17023367
21.981771803
21.83075038
20.266167375
22.626699143000003
19.95796942
21.756126053000003
21.62392063
20.053119625
22.411268893000003
17.464637609999997
19.242328742999998
19.14322757
17.567093565
19.778696083
17.565166859999998
19.375736243
19.23495707
17.675140815
20.007831582999998
17.74477961
19.532899742999998
19.38938207
17.844688565
20.167726333
19.65882467
21.458023553
21.31834663
19.752803125
22.100700143
19.75416792
21.558237303000002
21.41705863
19.843443125
22.201530143000003
19.97688092
21.765855303000002
21.614688880000006
20.048508625
22.397755143
20.236876811999995
22.044752945
21.903872772
20.325519767
22.690177035
20.294857812
22.094775945
21.953929271999996
20.380771766999995
22.740487535
18.508359515
20.307594398
20.166128225
18.60037447
20.946526238
18.698956765
20.490040148
20.348204225
18.795741719999995
21.128015738
19.115634015
20.868252148
20.726059975
19.18843447
21.508570738000003
18.02865911
19.805404493
19.66088482
18.136832815
20.437024333
17.944592609999994
19.725305993
19.58155632
18.052153065
20.357029333
17.779623609999994
19.535581243
19.39158332
17.848679315
20.031761083
18.614167002000002
20.392734635000004
20.248113962
18.714334707
21.024408475
18.476682502000003
20.257772385000006
20.114039962
18.577215457
20.891350975
19.703822265
21.471390898000003
21.326561225
19.79597897
22.103271238
19.510509265
21.271929898000003
21.127882975
19.59510722
21.905842238
19.120682265
20.872289398
20.729828225
19.19270997
21.511762738
18.792238297
19.290693362000003
20.64695593
20.510145007000002
21.19960102
18.751123547
19.246610112000003
20.60653943
20.460765007
21.169026769999995
19.87275931
20.369476375
21.721788193000002
21.58408452
22.274702033
19.77371406
20.266561625
21.599107193000002
21.46094052
22.247308783
19.56453756
20.046819375
21.380438942999998
21.24639802
22.043752283000003
17.072349499999998
17.580540064999994
18.909698632999998
18.76920471
19.471044972999998
17.1712975
17.679847314999996
18.993346883
18.86357246
19.550300223
17.351656
17.842483814999994
19.163619633000003
19.016388209999995
19.705991973
19.27646856
19.750059625
21.079090193
20.94025427
21.629906533000003
19.38008281
19.840753375
21.179888693
21.04398152
21.832636283000003
19.56439231
20.050754375
21.380191192999998
21.24077602
21.952044533000002
19.842174702
20.322503266999995
21.664878085
21.526172161999998
22.319306675
19.893973952000003
20.377949767
21.714920335
21.576222912
22.369531425
18.116930905000004
18.59938647
19.931186038
19.791927115
20.580048878
18.303420905000007
18.79426547
20.114177288
19.974668615
20.762511128
18.682001155
19.18639297
20.491265788
20.351802865
21.143168378
17.63724225
18.132735065
19.430800883000003
19.28909221
19.978902973
17.55337725
18.048753815
19.344651133
19.199835709999995
19.996533973
17.355749
17.845593065
19.159388133
19.02059521
19.706807223
18.215369642
18.711056707
20.018026775
19.876666352
20.662763615
18.077944642000002
18.574271707
19.881929275
19.741039352
20.527228365
19.296369655
19.819640719999995
21.106392788
20.955663615
21.751913378
14.27463400977778
21.306391062
22.61132638
22.470331206999997
23.26654222
18.686269405
19.18930672
20.494496288
20.354747365
21.145284628
''')).iloc[:,0])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/generatefeaturesforbinary/generated_03_29_13_54_45/target_histogram.png'
label = 'e_per_atom-sum(oxide_e_per_atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_target_histogram(y_df, savepath, label)
display(Image(filename='target_histogram.png'))
